In [2]:
from cmlreaders import CMLReader, get_data_index
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sb
import scipy
from scipy import stats
import pandas_to_pybeh as pb
from irasa.IRASA import IRASA

In [3]:
master = pd.read_excel(r'./220302_all_neuropsych.xlsx',engine="openpyxl") #read master spreadsheet of all RAM subjects demo/neuropsych data
sub_mat=list(np.zeros(len(master))*np.nan )
df = get_data_index('r1')
# df_select = pd.DataFrame([])
for sub in df['subject'].unique():
    num = int(sub[2:5])
    for i in master.index:
        sel = master.iloc[i]['Subject Number']
        if sel == num:
            sub_mat[i]=sub
# df_select[0:50]
master['subject']=sub_mat
master

,Subject Number,Study site,Gender,Years of Education,Age of seizure onset,Age at Implant (calculated),Phase II Implant Date,Phase III Implant Date,Traumatic Brain Injury (TBI) Notes,Date of Pre-operative Report,...,GAI,VCI,PRI,WMI,Pre-operative Report,Prior Neurosurgical Procedures (can select more than one),Prior Surgery: Resection Detail,Prior Surgery: Callosotomy Detail,Prior Surgery: Laser Ablation Detail,subject
0,1,University of Pennsylvania,Female,13.0,20.0,48.105026,2014-10-08,NaT,NaN,NaT,...,76.0,81,75,NaN,REP001_neurospych.pdf,NaN,NaN,NaN,NaN,R1001P
1,2,University of Pennsylvania,Female,16.0,34.0,49.383743,2014-11-12,NaT,NaN,NaT,...,84.0,89,84,NaN,R1002P_neuropsych.pdf,NaN,NaN,NaN,NaN,R1002P
2,3,University of Pennsylvania,Female,13.0,37.0,39.390382,2014-11-13,NaT,motor vehicle accident due to loss of consciou...,NaT,...,NaN,107,100,NaN,R1003P_neuropsych.pdf,NaN,NaN,NaN,NaN,R1003P
3,4,Dartmouth University,Female,12.0,6.0,51.918931,2014-11-06,NaT,"In early 90s, motor vehicle accident resulting...",NaT,...,NaN,78,82,NaN,R1004D Neuropsych.pdf,NaN,NaN,NaN,NaN,R1004D
4,5,University of Pennsylvania,Female,NaN,NaN,20.304432,2015-01-14,NaT,NaN,NaT,...,89.0,98,82,NaN,R1005P_neuropsych.pdf,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,597,Jefferson Hospital,Male,NaN,NaN,NaN,2021-12-10,NaT,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,598,Emory University,Female,12.0,16.0,39.568346,2021-12-15,NaT,NaN,2019-02-19,...,96.0,103,88,NaN,R1598E-Neuropsychologic Report_redacted.pdf,Laser ablation (i.e. Visualase),NaN,NaN,right anterior temporal hippocampus laser abla...,R1598E
598,599,UT Southwestern,Male,NaN,8.0,20.728808,2022-02-04,NaT,NaN,2019-06-03,...,101.0,98,104,NaN,NP report - 1599.pdf,Laser ablation (i.e. Visualase),NaN,NaN,Litt 1 2/2018 Left SMA Litt 2 11/2018 Mesial ...,NaN
599,600,UT Southwestern,Male,15.0,6.0,27.896534,2018-03-27,2022-02-07,NaN,2017-06-27,...,NaN,NaN,14,NaN,Neuropsych_R1600T.docx,"VNS implant,RNS implant (Neuropace)",NaN,NaN,NaN,NaN


In [4]:
subs = master.dropna(subset=['BAI', 'BDI', 'subject'])
len(subs)

162

In [5]:
## Find terciles for high/low BAI and BDI subsets
BAI = []
BDI = []
for i in master.index:
    sub_row = master.iloc[i]
    if pd.isnull(sub_row['BAI'])==False:
        BAI.append(sub_row['BAI'])
    if pd.isnull(sub_row['BDI'])==False:
        BDI.append(sub_row['BDI'])
    
# BAI_df = trial_df[trial_df['subject'].unique()].dropna('BAI')
# BDI_df = trial_df[trial_df['subject'].unique()].dropna('BDI')
BAI_low = np.quantile(BAI, float(1/3))
BAI_high = np.quantile(BAI, float(2/3))
BDI_low = np.quantile(BDI, float(1/3))
BDI_high = np.quantile(BDI, float(2/3))
print(BDI_low, BDI_high, BAI_low, BAI_high)

6.0 15.666666666666657 6.0 14.0


In [ ]:
eeg_df = pd.DataFrame([])

experiments = ['FR1', 'catFR1']
BAI_subs = master.dropna(subset=['subject','BAI'])
for sub in BAI_subs['subject']:
    try:
        for exp in experiments:
            localizations=df[(df['subject']==sub) & (df['experiment']==exp)]['localization'].unique()        

            for loc in localizations:
                montages = df[(df['subject']==sub) & (df['experiment']==exp) & (df['localization']==loc)]['montage'].unique()            

                for mont in montages: # iterate over montages            
                    data = df[np.logical_and(df["subject"] == sub, df['experiment']==exp)]
                    sessions=data[np.logical_and(data['montage']==mont,data['localization']==loc)]['session'].unique()

                    for sess in sessions: 
                        reader = CMLReader(sub, exp, sess, montage=mont, localization=loc)   #reader for loading CML data
                        pairs = reader.load("pairs")
                        events = reader.load('task_events')
                        word_events = events[events.type=='WORD']  # select encoding events 
                        rec_events = events[events.type=='REC_WORD']  # select retrieval events
                        eeg = reader.load_eeg(events = word_events, scheme = pairs, rel_start=300, rel_stop=1300).to_ptsa().mean('channel')
                        
                        eeg_df = eeg_df.append({ #create dataframe of events for analysis grouped by session
                            'subject':sub,
                            'session': sess,
                            'montage': mont,
                            'localization': loc,
                            'experiment': exp,
                            'eeg':eeg,
                            'BAI':BAI,
                            'BDI':BDI
                            },ignore_index=True)
                        
                        
    except:
        print('Failed for subject:', sub)
eeg_df[:100]

/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1001P/tal/R1001P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/experiments/FR1/sessions/0/ephys/current_processed/sources.json
/data/eeg/R1001P/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/

Failed for subject: R1006P


/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1010J/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1010J/tal/R1010J_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1010J/experiments/FR1/sessions/0/ephys/current_processed/sources.json
/data/eeg/R1010J/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1010J/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/

Failed for subject: R1093J


/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1098D/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1098D/tal/R1098D_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1098D/experiments/FR1/sessions/0/ephys/current_processed/sources.json
/data/eeg/R1098D/eeg.noreref/R1098D_FR1_0_31Dec15_1519.params.txt
/data/eeg/R1098D/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1098D/localizations/0

...

/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1348J/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1348J/tal/R1348J_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1348J/experiments/catFR1/sessions/1/ephys/current_processed/sources.json
/data/eeg/R1348J/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1350D/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/e

Failed for subject: R1350D


/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/readers/eeg.py:368: MissingChannelsWarning: The following channels are missing: 5Ldm1-5Ldm2, 5Ldm2-5Ldm3, 5Ldm3-5Ldm4, 5Ldm4-5Ldm5, 5Ldm5-5Ldm6, 5Ldm6-5Ldm7, 5Ldm7-5Ldm8, 5Ldm8-5Ldm9, 5Ldm9-5Ldm1, 25Ldm1-25Ldm2, 25Ldm2-25Ldm3, 25Ldm3-25Ldm4, 25Ldm4-25Ldm5, 25Ldm5-25Ldm6, 25Ldm6-25Ldm7, 25Ldm7-25Ldm8, 25Ldm8-25Ldm9, 25Ldm9-25Ldm1
  warnings.warn(msg, MissingChannelsWarning)


Failed for subject: R1354E
Failed for subject: R1366J
Failed for subject: R1370E
Failed for subject: R1379E


/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1383J/experiments/FR1/sessions/0/ephys/current_processed/sources.json
/data/eeg/R1383J/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1383J/experiments/catFR1/sessions/1/ephys/current_processed/sources.json
/data/eeg/R1383J/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/evsnyder/.conda/envs/cmlreader0/lib/python3.7/site-packages/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1383J/experiments/catFR1/sessions/2/ephys/current_processed/sources.json
/data/eeg/R1383J/eeg.noreref/para